In [20]:
from queue import deque
import numpy as np
import pickle
from keras import Input, Model
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Add, Activation
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import re
import os
from collections import Counter
from PIL import Image
import copy
from datetime import datetime
import csv
import matplotlib.pyplot as plt

In [21]:
from os.path import join
import os
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
PROJ = "My Drive/Project" # This is a custom path.
PROJECT_PATH = os.path.join(ROOT, PROJ)
IMAGEPATH=join(PROJECT_PATH, "images")
BOARDPATH=join(PROJECT_PATH, "boardresultscomp")
MODELPATH=join(PROJECT_PATH, "models")
COMPPATH=join(PROJECT_PATH, "modelscomp")
ACCEPTPATH=join(PROJECT_PATH, "modescomplate")
FILTERPATH=join(PROJECT_PATH, "modescomp3")
HUMANPATH=join(PROJECT_PATH, "modelscomphuman2")
RESULTPATH=join(PROJECT_PATH, "results")

In [23]:
action_space={'11':0,'12':1,'13':2,'14':3,'15':4,'21':5,'22':6,'23':7,'24':8,'25':9,'31':10,'32':11,'33':12,'34':13,'35':14,'41':15,'42':16,'43':17,'44':18,'45':19,'51':20,'52':21,'53':22,'54':23,'55':24,'1112':25,'1121':26,'1122':27,'1113':28,'1131':29,'1133':30,'1213':31,'1211':32,'1222':33,'1232':34,'1214':35,'1312':36,'1314':37,'1323':38,'1322':39,'1324':40,'1333':41,'1331':42,'1315':43,'1311':44,'1335':45,'1415':46,'1413':47,'1424':48,'1412':49,'1434':50,'1525':51,'1524':52,'1514':53,'1513':54,'1533':55,'1535':56,'2131':57,'2111':58,'2122':59,'2141':60,'2123':61,'2212':62,'2232':63,'2213':64,'2233':65,'2231':66,'2221':67,'2223':68,'2211':69,'2242':70,'2244':71,'2224':72,'2324':73,'2313':74,'2333':75,'2322':76,'2325':77,'2321':78,'2343':79,'2413':80,'2433':81,'2414':82,'2415':83,'2423':84,'2425':85,'2434':86,'2435':87,'2442':88,'2444':89,'2422':90,'2515':91,'2524':92,'2535':93,'2545':94,'2523':95,'3132':96,'3121':97,'3122':98,'3142':99,'3141':100,'3113':101,'3133':102,'3151':103,'3111':104,'3153':105,'3242':106,'3231':107,'3233':108,'3222':109,'3212':110,'3234':111,'3252':112,'3332':113,'3344':114,'3323':115,'3343':116,'3322':117,'3342':118,'3334':119,'3324':120,'3313':121,'3355':122,'3331':123,'3315':124,'3353':125,'3351':126,'3311':127,'3335':128,'3424':129,'3444':130,'3433':131,'3435':132,'3454':133,'3432':134,'3414':135,'3545':136,'3544':137,'3525':138,'3534':139,'3524':140,'3513':141,'3555':142,'3533':143,'3515':144,'3553':145,'4142':146,'4151':147,'4131':148,'4143':149,'4121':150,'4232':151,'4241':152,'4233':153,'4231':154,'4243':155,'4251':156,'4252':157,'4253':158,'4244':159,'4224':160,'4222':161,'4342':162,'4344':163,'4333':164,'4353':165,'4345':166,'4341':167,'4323':168,'4454':169,'4433':170,'4455':171,'4445':172,'4443':173,'4453':174,'4434':175,'4435':176,'4442':177,'4424':178,'4422':179,'4544':180,'4555':181,'4535':182,'4525':183,'4543':184,'5142':185,'5141':186,'5152':187,'5131':188,'5133':189,'5153':190,'5242':191,'5251':192,'5253':193,'5254':194,'5232':195,'5354':196,'5344':197,'5343':198,'5342':199,'5352':200,'5355':201,'5333':202,'5331':203,'5351':204,'5335':205,'5444':206,'5455':207,'5453':208,'5434':209,'5452':210,'5545':211,'5554':212,'5544':213,'5535':214,'5533':215,'5553':216}
reversed_action_space={v:k for k,v in action_space.items()}
action_list=list(action_space.keys())
bagh_moves_dict={(1,1):{(1,3),(3,1),(3,3)},(1,2):{(3,2),(1,4)},(1,3):{(3,3),(3,1),(1,5),(1,1),(3,5)},(1,4):{(1,2),(3,4)},(1,5):{(1,3),(3,3),(3,5)},(2,1):{(4,1),(2,3)},(2,2):{(4,2),(4,4),(2,4)},(2,3):{(2,5),(2,1),(4,3)},(2,4):{(4,2),(4,4),(2,2)},(2,5):{(4,5),(2,3)},(3,1):{(1,3),(3,3),(5,1),(1,1),(5,3)},(3,2):{(1,2),(3,4),(5,2)},(3,3):{(1,3),(5,5),(3,1),(1,5),(5,3),(5,1),(1,1),(3,5)},(3,4):{(5,4),(3,2),(1,4)},(3,5):{(1,3),(5,5),(3,3),(1,5),(5,3)},(4,1):{(4,3),(2,1)},(4,2):{(4,4),(2,4),(2,2)},(4,3):{(4,5),(4,1),(2,3)},(4,4):{(4,2),(2,4),(2,2)},(4,5):{(2,5),(4,3)},(5,1):{(3,1),(3,3),(5,3)},(5,2):{(5,4),(3,2)},(5,3):{(5,5),(3,3),(3,1),(5,1),(3,5)},(5,4):{(3,4),(5,2)},(5,5):{(3,5),(3,3),(5,3)}}
connected_points_dict={(1,1):{(1,2),(2,1),(2,2)},(1,2):{(1,3),(1,1),(2,2)},(1,3):{(1,2),(1,4),(2,3),(2,2),(2,4)},(1,4):{(1,5),(1,3),(2,4)},(1,5):{(2,5),(2,4),(1,4)},(2,1):{(3,1),(1,1),(2,2)},(2,2):{(1,2),(3,2),(1,3),(3,3),(3,1),(2,1),(2,3),(1,1)},(2,3):{(2,4),(1,3),(3,3),(2,2)},(2,4):{(1,3),(3,3),(1,4),(1,5),(2,3),(2,5),(3,4),(3,5)},(2,5):{(1,5),(2,4),(3,5)},(3,1):{(3,2),(2,1),(2,2),(4,2),(4,1)},(3,2):{(4,2),(3,1),(3,3),(2,2)},(3,3):{(3,2),(4,4),(2,3),(4,3),(2,2),(4,2),(3,4),(2,4)},(3,4):{(2,4),(4,4),(3,3),(3,5)},(3,5):{(4,5),(4,4),(2,5),(3,4),(2,4)},(4,1):{(4,2),(5,1),(3,1)},(4,2):{(3,2),(4,1),(3,3),(3,1),(4,3),(5,1),(5,2),(5,3)},(4,3):{(4,2),(4,4),(3,3),(5,3)},(4,4):{(5,4),(3,3),(5,5),(4,5),(4,3),(5,3),(3,4),(3,5)},(4,5):{(4,4),(5,5),(3,5)},(5,1):{(4,2),(4,1),(5,2)},(5,2):{(4,2),(5,1),(5,3)},(5,3):{(5,4),(4,4),(4,3),(4,2),(5,2)},(5,4):{(4,4),(5,5),(5,3)},(5,5):{(4,5),(5,4),(4,4)}}

In [24]:
'''
FEN -> Similar to Forsyth–Edwards Notation in chess
    Starting fen => "B3B/5/5/5/B3B G 0"
        {"B3B/5/5/5/B3B"(similar to chess,"B" - Bagh, "G" - Goat)}
        {"G" or "B" represents who has next move}
        {"0" number of moves by goat}
PGN -> Portable Game Notation like in chess
        Move to move tracking notation
        <Move number>. G(<old_position>)<new_position> (...B<old_position><new_position>)
        Example : 1.G33 B1122 2.G44
        [ Note: G<new_position> for unplaced piece ]
'''


def render_points(p): return (103*(p[1]-1), 103*(p[0]-1))  # for pillow image


BOARD_IMG = Image.open(os.path.join(IMAGEPATH, 'board.png'), 'r')
BAGH_SPRITE = Image.open(os.path.join(IMAGEPATH, 'bagh.png'), 'r')
GOAT_SPRITE = Image.open(os.path.join(IMAGEPATH, 'goat.png'), 'r')


class Board:

    def __init__(self, description=""):
        self.reset()
        if description.strip():
            self.pgn_converter(description.strip())

    def __getitem__(self, index):
        return self.board[index[0] - 1][index[1] - 1]

    def __setitem__(self, index, value):
        self.board[index[0] - 1][index[1] - 1] = value

    @property
    def no_of_moves_made(self):
        return self.no_of_goat_moves + self.no_of_bagh_moves

    def _possible_goat_moves(self):
        # the function is independent of whose turn it is to play, use at your own risk.
        move_list = set()
        if self.no_of_goat_moves < 20:
            return {f'G{x1}{y1}' for x1 in range(1, 6) for y1 in range(1, 6) if
                    (x1, y1) not in self.bagh_points.union(self.goat_points)}
        else:
            for x1, y1 in self.goat_points:
                move_list.update({f'G{x1}{y1}{x2}{y2}' for x2, y2 in self[x1, y1].valid_moves()})
            return move_list

    def _possible_bagh_moves(self):
        # the function is independent of whose turn it is to play, use at your own risk.
        move_list = set()
        for x1, y1 in self.bagh_points:
            move_list.update({f'B{x1}{y1}{x2}{y2}' for x2, y2 in self[x1, y1].valid_non_special_moves()})
            move_list.update({f'Bx{x1}{y1}{x2}{y2}' for x2, y2 in self[x1, y1].valid_bagh_moves()})
        return move_list

    def possible_moves(self):
        if self.is_game_over():
            return 0
        if self.next_turn == "G":
            return self._possible_goat_moves()
        else:
            return self._possible_bagh_moves()

    def possible_moves_vector(self):
        moves_vector = np.zeros(217)
        if self.is_game_over():
            return moves_vector
        if self.next_turn == "G" and self.no_of_goat_moves < 20:
            for x1 in range(1, 6):
                for y1 in range(1, 6):
                    if (x1, y1) not in self.bagh_points.union(self.goat_points):
                        moves_vector[action_space[f'{x1}{y1}']] = 1
        elif self.next_turn == "G" and self.no_of_goat_moves >= 20:
            for x1, y1 in self.goat_points:
                for x2, y2 in self[x1, y1].valid_moves():
                    moves_vector[action_space[f'{x1}{y1}{x2}{y2}']] = 1
        else:
            for x1, y1 in self.bagh_points:
                for x2, y2 in self[x1, y1].valid_moves():
                    moves_vector[action_space[f'{x1}{y1}{x2}{y2}']] = 1
        return moves_vector

    def pgn_converter(self, pgn):
        move_list = re.findall(
            r'[0-9]+\.\s*([G][1-5]{2,4})\s*([B][x]?[1-5]{4})?', pgn)
        for moves in move_list:
            for move in moves:
                if move == "":
                    break
                self.move(move)

    def fen_to_board(self, fen):
        rows = fen.split(" ")[0].split("/")
        for x in range(5):
            counter = 1
            for y in rows[x]:
                if y == "B":
                    Bagh(self, (x + 1, counter))
                    counter += 1
                elif y == "G":
                    Bagh(self, (x + 1, counter))
                    counter += 1
                else:
                    for _ in range(int(y)):
                        counter += 1

    @property
    def baghs_trapped(self):
        counter = 0
        for bagh in self.bagh_points:
            if not self[bagh[0], bagh[1]].valid_moves():
                counter += 1
        return counter

    @property
    def all_goats_trapped(self):
        return self.next_turn == "G" and not self._possible_goat_moves()

    def show_info(func):
        def wrapper(self):
            if self.no_of_moves_made:
                print(f"Last move: {self.moves[-1]}")
            print(
                f"Goats Placed: {self.goats_placed}, Goats Captured: {self.goats_captured}, Baghs Trapped: {self.baghs_trapped}")
            func(self)
            if self.is_game_over():
                print("Game over.")
                if self.winner():
                    print(f"Winner : {self.winner()}")
                else:
                    print(f"The game is a draw.")
                return
            print(f"{self.next_turn} to play.")
            print(f"Possible moves:", end="")
            for move in self.possible_moves():
                print(f" {move}", end="")
            print()
            print()
        return wrapper

    @show_info
    def show_board(self):
        rep1 = ''' ¦ ＼         ¦         ／ ¦ ＼         ¦         ／ ¦    
 ¦   ＼       ¦       ／   ¦   ＼       ¦       ／   ¦    
 ¦     ＼     ¦     ／     ¦     ＼     ¦     ／     ¦    
 ¦       ＼   ¦   ／       ¦       ＼   ¦   ／       ¦    
 ¦         ＼ ¦ ／         ¦         ＼ ¦ ／         ¦    '''
        rep2 = ''' ¦         ／ ¦ ＼         ¦          ／¦ ＼         ¦    
 ¦       ／   ¦   ＼       ¦        ／  ¦   ＼       ¦    
 ¦     ／     ¦     ＼     ¦      ／    ¦     ＼     ¦    
 ¦   ／       ¦       ＼   ¦    ／      ¦       ＼   ¦    
 ¦ ／         ¦         ＼ ¦  ／        ¦         ＼ ¦    '''
        print(
            f"[{self[1,1]}]11--------[{self[1,2]}]12--------[{self[1,3]}]13--------[{self[1,4]}]14--------[{self[1,5]}]15")
        print(rep1)
        print(
            f"[{self[2,1]}]21--------[{self[2,2]}]22--------[{self[2,3]}]23--------[{self[2,4]}]24--------[{self[2,5]}]25")
        print(rep2)
        print(
            f"[{self[3,1]}]31--------[{self[3,2]}]32--------[{self[3,3]}]33--------[{self[3,4]}]34--------[{self[3,5]}]35")
        print(rep1)
        print(
            f"[{self[4,1]}]41--------[{self[4,2]}]42--------[{self[4,3]}]43--------[{self[4,4]}]44--------[{self[4,5]}]45")
        print(rep2)
        print(
            f"[{self[5,1]}]51--------[{self[5,2]}]52--------[{self[5,3]}]53--------[{self[5,4]}]54--------[{self[5,5]}]55")

    def validate_placement(self, move):
        x1, y1 = int(move[1]), int(move[2])
        self.validate_points(move, x1, y1)
        if not self.goats_placed < 20:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} More than 20 goats cannot be placed")
        if self[x1, y1]:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} The coordinate is already occupied.")
        return True

    def validate(self, move):
        if self.is_game_over():
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} The game is already over.")
        move = move.strip()
        if len(move) not in {3, 5, 6}:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Error ! Could not recognise the move.")
        if move[0] != self.next_turn:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Illegal Move.It is other side's turn.")
        if move[:2] == "Bx":
            return self.validate_capture(move)
        if len(move) == 3:
            if self.goats_placed >= 20:
                raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Futher piece cannot be placed.")
            if move[0] == "B":
                raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Further Bagh cannot be placed.")
            return self.validate_placement(move)
        if move[0] == "G" and len(move) == 5 and self.no_of_goat_moves < 20:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} All the goats must be placed first.")
        if move[:2] == "Gx":
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Goats cannot capture.")
        x1, y1, x2, y2 = int(move[1]), int(move[2]), int(move[3]), int(move[4])
        self.validate_points(move, x1, y1, x2, y2)
        self.validate_pp(move, x1, y1, move[0])

        if move[0] == "G":
            if not ((x2, y2) in self[x1, y1].valid_moves()):
                raise Exception(
                    f"{(self.no_of_moves_made+2)//2}.{move} is not a valid move.")
        elif move[0] == "B":
            if not ((x2, y2) in self[x1, y1].valid_non_special_moves()):
                raise Exception(
                    f"{(self.no_of_moves_made+2)//2}.{move} is not a valid move.")
        return True

    def validate_capture(self, move):
        x1, y1, x2, y2 = int(move[2]), int(move[3]), int(move[4]), int(move[5])
        self.validate_points(move, x1, y1, x2, y2)
        self.validate_pp(move, x1, y1, move[0])
        if not ((x2, y2) in self[x1, y1].valid_bagh_moves()):
            raise Exception(
                f"{(self.no_of_moves_made+2)//2}.{move} is not a valid move.")
        return True

    def validate_points(self, move, x1, y1, x2=1, y2=1):
        if not (0 < x1 < 6 and 0 < y1 < 6 and 0 < x2 < 6 and 0 < y2 < 6):
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Invalid PGN. Coordinates not in range.")

    def validate_pp(self, move, x1, y1, p):
        if not self[x1, y1]:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} ({x1},{y1}) is not occupied.")
        if self[x1, y1].__str__() != p:
            raise Exception(f"{(self.no_of_moves_made+2)//2}.{move} Piece at ({x1},{y1}) is other than specified.")

    def safe_move(self, move):
        if len(move) == 3:
            Goat(self, (int(move[1]), int(move[2])))
            self.no_of_goat_moves += 1
            self.goats_placed += 1
        else:
            if len(move) == 5:
                x1, y1, x2, y2 = int(move[1]), int(
                    move[2]), int(move[3]), int(move[4])
            elif len(move) == 6:
                x1, y1, x2, y2 = int(move[2]), int(
                    move[3]), int(move[4]), int(move[5])
                x3, y3 = (x1 + x2) // 2, (y1 + y2) // 2
                self[x3, y3] = 0
                self.goat_points.remove((x3, y3))
                self.goats_captured += 1
            self[x1, y1] = 0
            if move[0] == "G":
                self.goat_points.remove((x1, y1))
                Goat(self, (x2, y2))
                self.no_of_goat_moves += 1
            elif move[0] == "B":
                self.bagh_points.remove((x1, y1))
                Bagh(self, (x2, y2))
                self.no_of_bagh_moves += 1

        self.moves.append(move)
        pgn_update = ""
        if self.next_turn == "G":
            pgn_update += f"{self.no_of_goat_moves}. "
        pgn_update += move
        self.pgn += " " + pgn_update
        self.next_turn = "G" if self.next_turn == "B" else "B"

        self.fen = self.board_to_fen()
        self.fen_history.append(self.fen)
        if self.no_of_goat_moves >= 20:
            self.fen_count.update([self.fen.split(" ")[0]])

        if self.is_game_over():
            if self.winner() == "B":
                self.pgn += "# 0-1"
            elif self.winner() == "G":
                self.pgn += "# 1-0"
            else:
                self.pgn += "# 1/2-1/2"

    def board_to_fen(self):
        string = ""
        for x in range(1, 6):
            counter = 0
            for y in range(1, 6):
                if self[x, y]:
                    counter = 0
                    string += self[x, y].__str__()
                else:
                    counter += 1
                    if y == 5 or self[x, y + 1] != 0:
                        string += str(counter)
            string += "/"
        return f"{string[:-1]} {self.next_turn} {self.no_of_goat_moves}"

    def move(self, move):
        if self.validate(move):
            self.safe_move(move)

    def pure_move(self, move):
        if len(move) == 2:
            self.move(f"G{move}")
        else:
            x1, y1, x2, y2 = move
            if (int(x1) - int(x2))**2 + (int(y1) - int(y2))**2 <= 2:
                self.move(f"{self.next_turn}{move}")
            else:
                self.move(f'{self.next_turn}x{move}')

    def is_game_over(self):
        if self.goats_captured >= 5 or self.baghs_trapped == 4 or self.check_draw() or self.all_goats_trapped:
            return True
        return False

    def check_draw(self):
        if max(self.fen_count.values()) >= 3:
            return 1
        return 0

    def winner(self):
        if self.goats_captured >= 5 or self.all_goats_trapped:
            return "B"
        if self.baghs_trapped == 4:
            return "G"
        if self.check_draw():
            return 0
        raise Exception("Game is not over yet !")

    def fen_state(self, fen):
        state = np.zeros((2, 5, 5))
        rows = fen.split(" ")[0].split("/")
        for x in range(5):
            counter = 0
            for y in rows[x]:
                if y == "G":
                    state[0, x, counter] = 1
                    counter += 1
                elif y == "B":
                    state[1, x, counter] = 1
                    counter += 1
                else:
                    for _ in range(int(y)):
                        counter += 1
        return state

    def board_repr(self):
        state = np.zeros((5, 5, 5))
        state[[0, 1]] = self.fen_state(self.fen)
        state[2, :, :] = self.goats_captured
        state[3, :, :] = self.baghs_trapped
        if self.next_turn == "G":
            state[4, :, :] = 1
        return state

    def reset(self):
        self.board = [[0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0]]
        self.next_turn = "G"
        self.no_of_goat_moves = 0
        self.no_of_bagh_moves = 0
        self.goats_placed = 0
        self.goats_captured = 0
        self.goat_points = set()
        self.bagh_points = set()
        self.fen = "B3B/5/5/5/B3B G 0"
        self.fen_history = [self.fen]
        self.fen_count = Counter([self.fen.split(" ")[0]])
        self.pgn = ""
        self.moves = list()
        self.fen_to_board(self.fen)

    def recent_player(self):
        return "G" if self.no_of_moves_made % 2 else "B"

    def undo(self, no_of_moves=1):
        if no_of_moves > self.no_of_moves_made:
            raise Exception(
                "The number of moves to undo is greater than the number of moves made in the board.")
        move_list = self.moves
        n = self.no_of_moves_made - no_of_moves
        self.reset()
        for move in move_list:
            if move == "" or n == 0:
                return move_list[-no_of_moves:]
            self.move(move)
            n -= 1

    @show_info
    def lightweight_show_board(self):
        print("-" * 26)
        for row in self.board:
            for x in row:
                if x:
                    print(f"| {x.__str__()} ", end=" ")
                else:
                    print("|   ", end=" ")
            print("|")
            print("-" * 26)

    def render(self):
        img = Image.new("RGBA", (480, 480), (255, 255, 255, 0))
        img.paste(BOARD_IMG, (0, 0))
        for point in self.goat_points:
            img.paste(GOAT_SPRITE, render_points(point), mask=GOAT_SPRITE)
        for point in self.bagh_points:
            img.paste(BAGH_SPRITE, render_points(point), mask=BAGH_SPRITE)
        display(img)
        now = datetime.now()
        filename='board'+ str(now) +'.png'
        img.save(join(BOARDPATH, filename))


class Piece:

    def __init__(self, board, position=0):
        if position:
            if not (1, 1) <= position <= (5, 5):
                raise Exception(f"Invalid Coordinate for {self.__repr__()} - {position}")
            if board[position[0], position[1]]:
                raise Exception(
                    f"Cannot place {self.__repr__()} at coordinate {position} occupied by {board[position[0],position[1]].__repr__()}")
        self.position = position
        self.board = board
        self.board[position[0], position[1]] = self
        self.x=131.2+(position[1]-1)*103+30
        self.y=114.5+(position[0]-1)*103+30

    def connected_points(self):
        if self.position:
            return connected_points_dict[self.position]
        else:
            return 0

    def valid_moves(self):
        return {x for x in self.connected_points()
                if not self.board[x[0], x[1]]}


class Bagh(Piece):

    def __init__(self, board, position):
        super(Bagh, self).__init__(board, position)
        self.board.bagh_points.add(position)

    def __str__(self):
        return "B"

    def __repr__(self):
        return "Bagh"

    def special_connected_points(self):
        return bagh_moves_dict[self.position]

    def valid_bagh_moves(self):
        return {x for x in self.special_connected_points()
                if (not self.board[x[0], x[1]]) and self.board[
                    (x[0] + self.position[0]) // 2, (x[1] + self.position[1]) // 2].__class__ == Goat}

    def valid_moves(self):
        return super(Bagh, self).valid_moves().union(self.valid_bagh_moves())

    def valid_non_special_moves(self):
        return super(Bagh, self).valid_moves()


class Goat(Piece):

    def __init__(self, board, position):
        super(Goat, self).__init__(board, position)
        self.board.goat_points.add(position)

    def __str__(self):
        return "G"

    def __repr__(self):
        return "Goat"


In [25]:
def softmax(x):
    arr=np.exp(x)
    return arr/sum(arr)


def mask_illegal(policy_vector, possible_moves):
    # returns new softmax
    valid_policy_vector = policy_vector * possible_moves

    # if all legal positions are masked, return equal prob for all possible moves
    if not valid_policy_vector.any():
        return possible_moves/np.sum(possible_moves)

    return softmax(np.log(valid_policy_vector+1e-10))


def symmetry_board_moves(inputs):
    ''' input -> [(board_repr(5,5,5),pi_vector,value),......]
    output - > [(board_repr(5,5,5),pi_vector,value),......]
    Each input has 8 symmetries including the given input i.e. rotation(0,90,180,270)+flip_horizontal(rotation(0,90,180,270))
    These 8 output for single input encapsulates all rotation and flip horizontally or vertically.
    '''
    a = np.empty((5, 5), dtype="<U2")
    for x in range(1, 6):
        for y in range(1, 6):
            a[x - 1, y - 1] = f'{x}{y}'
    output = []
    for board_repr, move_probs, value in inputs:
        moves2 = []
        probs = move_probs[move_probs != 0]
        for c in range(217):
            if move_probs[c]:
                moves2.append(reversed_action_space[c])
        for i in range(4):
            moves_after = []
            tracker = np.rot90(a, i)
            for move2 in moves2:
                if len(move2) == 2:
                    l = np.where(tracker == move2)
                    moves_after.append(f'{l[0][0]+1}{l[1][0]+1}')
                if len(move2) == 4:
                    l1 = np.where(tracker == move2[:2])
                    l2 = np.where(tracker == move2[2:])
                    moves_after.append(f'{l1[0][0]+1}{l1[1][0]+1}{l2[0][0]+1}{l2[1][0]+1}')
            new_l = [action_space[i] for i in moves_after]
            new_move_probs = np.zeros(217)
            new_move_probs[new_l] = probs
            output.append((np.array([np.rot90(board_repr[0], i), np.rot90(board_repr[1], i), *board_repr[2:]]),
                           new_move_probs, value))
        for i in range(4):
            moves_after = []
            tracker = np.fliplr(np.rot90(a, i))
            for move2 in moves2:
                if len(move2) == 2:
                    l = np.where(tracker == move2)
                    moves_after.append(f'{l[0][0]+1}{l[1][0]+1}')
                if len(move2) == 4:
                    l1 = np.where(tracker == move2[:2])
                    l2 = np.where(tracker == move2[2:])
                    moves_after.append(f'{l1[0][0]+1}{l1[1][0]+1}{l2[0][0]+1}{l2[1][0]+1}')
            new_l = [action_space[i] for i in moves_after]
            new_move_probs = np.zeros(217)
            new_move_probs[new_l] = probs
            output.append((np.array(
                [np.fliplr(np.rot90(board_repr[0], i)), np.fliplr(np.rot90(board_repr[1], i)), *board_repr[2:]]),
                           new_move_probs, value))
    return output

In [26]:
action_list=np.array(action_list)

class PolicyValueNet:
    def __init__(self, model_file=None):
        self.config = ModelConfig()

        if model_file:
            self.model = load_model(model_file, compile=True)
        else:
            self.create()
            self.loss_train_op()

    def create(self):

        in_x = x = Input((5, 5, 5))

        # (batch, channels, height, width)
        x = Conv2D(filters=self.config.cnn_first_filter_num, kernel_size=self.config.cnn_first_filter_size, padding="same",
                   data_format="channels_first", use_bias=False, kernel_regularizer=l2(self.config.l2_reg),
                   name=f"input1_conv-{self.config.cnn_filter_num}-{self.config.cnn_first_filter_size}x{self.config.cnn_first_filter_size}")(
            x)
        x = Conv2D(filters=self.config.cnn_filter_num, kernel_size=self.config.cnn_first_filter_size, padding="same",
                   data_format="channels_first", use_bias=False, kernel_regularizer=l2(self.config.l2_reg),
                   name=f"input_conv-{self.config.cnn_filter_num}-{self.config.cnn_first_filter_size}x{self.config.cnn_first_filter_size}")(
            x)
        x = BatchNormalization(axis=1, name="input_batchnorm")(x)
        x = Activation("relu", name="input_relu")(x)

        for i in range(self.config.resnet_N):
            x = self.build_residual_block(x, i + 1)

        res_out = x

        # for policy output
        x = Conv2D(filters=4, kernel_size=1, data_format="channels_first", use_bias=False,
                   kernel_regularizer=l2(self.config.l2_reg), name="policy_conv-4-1x1")(res_out)
        x = BatchNormalization(axis=1, name="policy_batchnorm")(x)
        x = Activation("relu", name="policy_relu")(x)
        x = Flatten(name="policy_flatten")(x)
        ''' For action policy network output, BaghChal has 217 moves possible at any time period
         (For more: lookup_table.py ->action_space)'''
        policy_out = Dense(217, kernel_regularizer=l2(
            self.config.l2_reg), activation="softmax", name="policy_out")(x)

        # for value output
        x = Conv2D(filters=2, kernel_size=1, data_format="channels_first", use_bias=False,
                   kernel_regularizer=l2(self.config.l2_reg), name="value_conv-2-1x1")(res_out)
        x = BatchNormalization(axis=1, name="value_batchnorm")(x)
        x = Activation("relu", name="value_relu")(x)
        x = Flatten(name="value_flatten")(x)
        x = Dense(self.config.value_dense_size, kernel_regularizer=l2(
            self.config.l2_reg), activation="relu", name="value_dense")(x)
        value_out = Dense(1, kernel_regularizer=l2(
            self.config.l2_reg), activation="tanh", name="value_out")(x)

        self.model = Model(
            in_x, [policy_out, value_out], name="BaghChal_model")

    def build_residual_block(self, x, index):
        in_x = x
        res_name = "resnet" + str(index)
        x = Conv2D(filters=self.config.cnn_filter_num, kernel_size=self.config.cnn_filter_size, padding="same",
                   data_format="channels_first", use_bias=False, kernel_regularizer=l2(self.config.l2_reg),
                   name=f"{res_name}_conv1-{self.config.cnn_filter_num}-{self.config.cnn_filter_size}x{self.config.cnn_filter_size}")(
            x)
        x = BatchNormalization(axis=1, name=f"{res_name}_batchnorm1")(x)
        x = Activation("relu", name=f"{res_name}_relu1")(x)
        x = Conv2D(filters=self.config.cnn_filter_num, kernel_size=self.config.cnn_filter_size, padding="same",
                   data_format="channels_first", use_bias=False, kernel_regularizer=l2(self.config.l2_reg),
                   name=f"{res_name}_conv2-{self.config.cnn_filter_num}-{self.config.cnn_filter_size}x{self.config.cnn_filter_size}")(
            x)
        x = BatchNormalization(axis=1, name=f"{res_name}_batchnorm2")(x)
        x = Add(name=f"{res_name}_add")([in_x, x])
        x = Activation("relu", name=f"{res_name}_relu2")(x)
        return x

    def policy_value(self, state_input):
        state_input = np.array(state_input)  # _on_batch(state_input_union)
        results = self.model.predict(state_input)
        return results

    def policy_value_fn(self, board4):
        """
        input: board
        output: a list of (action, probability) tuples for each available action and the score of the board state
        """
        legal_positions = board4.possible_moves_vector()
        act_probs_raw, value = self.policy_value(np.expand_dims(board4.board_repr(),0))
        act_probs_raw = mask_illegal(act_probs_raw[0], legal_positions)
        index_list=np.where(legal_positions == 1)[0]
        actions=action_list[index_list]
        act_probs=act_probs_raw[index_list]
        return zip(actions,act_probs), value[0]

    def loss_train_op(self):
        """
        Three loss terms：
        loss = (z - v)^2 + pi^T * log(p) + c||theta||^2
        """
        # get the train op
        opt = Adam()
        losses = ['categorical_crossentropy', 'mean_squared_error']
        self.model.compile(optimizer=opt, loss=losses)

    def save_model(self, model_filename):
        """ save model to file """
        self.model.save(model_filename)

    def train(self,board_repr2,mtcs_prob,winner,epochs):
        board_repr2 = np.array(board_repr2)
        mtcs_prob = np.array(mtcs_prob)
        winner = np.array(winner)
        self.model.fit(board_repr2, [mtcs_prob, winner],epochs=epochs)



In [27]:
class Game:
    def __init__(self, board5=Board()):
        self.board5 = board5         

    def start_play(self, GoatPlayer, BaghPlayer, show=True):
        self.board5=Board()
        states=[]
        mcts_probs=[]
        value=[]
        """start a game between two players"""
        if show:
            self.board5.lightweight_show_board()
            self.board5.render()
        while True:
            player_to_move = GoatPlayer if self.board5.next_turn == "G" else BaghPlayer
            idle_player = BaghPlayer if player_to_move == GoatPlayer else GoatPlayer
            states.append(self.board5.board_repr())
            player_index = 1 if self.board5.next_turn == "G" else -1
            value.append(player_index)
            if player_to_move.__class__==HumanPlayer:
                move3 = player_to_move.get_action(self.board5)
                a=np.zeros(217)
                if len(move3)==3:
                    a[action_space[move3[1:]]]=1
                else:
                    a[action_space[move3[-4:]]]=1
                mcts_probs.append(a)
                self.board5.safe_move(move3)
                idle_player.update_move(move3)
            else:
                move3, a = player_to_move.get_action(self.board5, temp=0.8, return_prob=1)
                if idle_player.__class__!=HumanPlayer:
                    idle_player.update_move(move3)
                self.board5.pure_move(move3)
                mcts_probs.append(a)
            if show:
                self.board5.lightweight_show_board()
                self.board5.render()
            end = self.board5.is_game_over()
            if end:
                value=np.array(value)
                if self.board5.winner() == "B":
                    value *= -1
                elif self.board5.winner() == 0:
                    value *= 0
                player.reset_player()
                return zip(states, mcts_probs, value)

    def start_self_play(self, player, show=True, temp=1e-3,greedy_player=None,who_greedy=""):
        """ start a self-play game using a MCTS player, reuse the search tree,
        and store the self-play data: (state, mcts_probs, z) for training
        """
        self.board5=Board()
        states, mcts_probs, value = [], [], []
        while True:
            if greedy_player:
                if who_greedy=="B":
                    player_to_move = player if self.board5.next_turn == "G" else greedy_player
                elif who_greedy=="G":
                    player_to_move = player if self.board5.next_turn == "B" else greedy_player
                else:
                    player_to_move=greedy_player
            else:
                player_to_move = player
            if greedy_player:
                idle_player = player if player_to_move == player else greedy_player
            move3, move_probs = player_to_move.get_action(self.board5,
                                                 temp=temp,
                                                 return_prob=1)
            if greedy_player:
                idle_player.update_move(move3)
            # store the data
            states.append(self.board5.board_repr())
            mcts_probs.append(move_probs)

            # temporarily store 1 for goat move and -1 for bagh
            # later can multiply,ie if goat is winner <value>*1
            # if bagh wins <value>*-1 , if draw *0
            player_index = 1 if self.board5.next_turn == "G" else -1
            value.append(player_index)

            # perform a move
            self.board5.pure_move(move3)
            if show:
                self.board5.lightweight_show_board()
                # self.board5.render()
            end = self.board5.is_game_over()
            if end:
                value=np.array(value)
                # winner from the perspective of the current player of each state
                if self.board5.winner() == "B":
                    value *= -1
                elif self.board5.winner() == 0:
                    value *= 0
                # reset MCTS root node
                player.reset_player()
                w=self.board5.winner()
                return w, zip(states, mcts_probs, value)

    def play_games(self, GoatPlayer, BaghPlayer, show=True, temp=1e-3):
        """ start a self-play game using a MCTS player, reuse the search tree,
        and store the self-play data: (state, mcts_probs, z) for training
        """
        self.board5=Board()
        states, mcts_probs, value = [], [], []
        while True:
            player_to_move = GoatPlayer if self.board5.next_turn == "G" else BaghPlayer
            idle_player = BaghPlayer if player_to_move == GoatPlayer else GoatPlayer

            move3, move_probs = player_to_move.get_action(self.board5,
                                                 temp=temp,
                                                 return_prob=1)
            
            idle_player.update_move(move3)
            # store the data
            states.append(self.board5.board_repr())
            mcts_probs.append(move_probs)

            # temporarily store 1 for goat move and -1 for bagh
            # later can multiply,ie if goat is winner <value>*1
            # if bagh wins <value>*-1 , if draw *0
            player_index = 1 if self.board5.next_turn == "G" else -1
            value.append(player_index)

            # perform a move
            self.board5.pure_move(move3)
            if show:
                self.board5.lightweight_show_board()
                # self.board5.render()
            end = self.board5.is_game_over()
            if end:
                value=np.array(value)
                # winner from the perspective of the current player of each state
                if self.board5.winner() == "B":
                    value *= -1
                elif self.board5.winner() == 0:
                    value *= 0
                # reset MCTS root node
                player_to_move.reset_player()
                idle_player.reset_player()
                w=self.board5.winner()
                return w, zip(states, mcts_probs, value)


In [28]:
class ModelConfig:
    def __init__(self):
        self.cnn_first_filter_num=128
        self.cnn_filter_num = 64
        self.cnn_first_filter_size = 3
        self.cnn_filter_size = 2
        self.resnet_N = 5
        self.l2_reg = 1e-4
        self.value_dense_size = 32

class TrainConfig:
    def __init__(self):
        self.learn_rate = 2e-3
        self.lr_multiplier = 1.0  # adaptively adjust the learning rate based on KL
        self.temp = 0.8  # the temperature param
        self.n_playout = 10  # num of simulations for each move
        self.c_puct = 0.001
        self.buffer_size = 10000
        self.batch_size = 1  # mini-batch size for training

        self.play_batch_size = 5 # how many games to play
        self.epochs = 20  # num of train_steps for each update
        self.ld=0.8

        #c_puct=0.0001 ld =0.8
class TrainGreedyConfig:
    def __init__(self):
        self.learn_rate = 2e-3
        self.lr_multiplier = 1.0  # adaptively adjust the learning rate based on KL
        self.temp = 0.8  # the temperature param
        self.n_playout = 10  # num of simulations for each move
        self.c_puct = 1
        self.buffer_size = 20000
        self.batch_size = 1  # mini-batch size for training

        self.play_batch_size = 1 # how many games to play
        self.epochs = 10  # num of train_steps for each update
        self.ld=0.8

        #c_puct=0.0001 ld =0.8

class PlayConfig:
    def __init__(self):
        self.learn_rate = 2e-3
        self.lr_multiplier = 1.0  # adaptively adjust the learning rate based on KL
        self.temp = 0.8  # the temperature param
        self.n_playout = 10  # num of simulations for each move
        self.c_puct = 5
        self.buffer_size = 20000
        self.batch_size = 1  # mini-batch size for training

        self.play_batch_size = 1 # how many games to play
        self.epochs = 10  # num of train_steps for each update
        self.ld=0.8

        #c_puct=0.0001 ld =0.8


In [29]:
class Node:
    def __init__(self, parent, prior_prob):
        self.parent = parent
        self.children = {}
        self.prior_prob = prior_prob
        self.W = 0
        self.n_visits = 0

    @property
    def Q(self):
        return self.W / self.n_visits if self.n_visits != 0 else 0

    def get_value(self, cpuct):
        self.U = cpuct * self.prior_prob * \
                 (self.n_visits ** 0.5 / (1 + self.parent.n_visits))
        return self.Q + self.U

    def expand(self, action_prob2):
        for action2, prob2 in action_prob2:
            if action2 not in self.children:
                self.children[action2] = Node(self, prob2)

    def update(self, value):
        self.n_visits += 1
        self.W += value

    def update_ancestors(self, value):
        if self.parent:
            self.parent.update(-value)
        self.update(value) 

    def select(self, cpuct, sensible_moves):
        # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.children.items()]
        # tempchildren = {}
        # print (sensible_moves)
        # for action11 in sensible_moves:
        #     print(action11[1:]) 
        #     if action11[1:] in self.children:
                # print("here")
                # print(self.children[action[1:]])
                # tempchildren[action11[1:]]=self.children[action11[1:]]
        # [print(act, f"{node.Q} {node.n_visits}") for act, node in tempchildren.items()]
        # return max(tempchildren.items(),
        #            key=lambda act_node: act_node[1].get_value(cpuct))
        return max(self.children.items(),
                   key=lambda act_node: act_node[1].get_value(cpuct))

    def is_leaf(self):
        return self.children == {}

    def is_root(self):
        return self.parent is None


class MCTS:
    def __init__(self, policy_value_fn, cpuct, n_playout):
        self.root = Node(None, 1)
        self.policy = policy_value_fn
        self.cpuct = cpuct
        self.n_playout = n_playout

    def playout(self, board2):
        node2 = self.root
        # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.root.children.items()]
        # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.root.children.items()]
        # print("here2")
        while True:
            if node2.is_leaf():
                break
            # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.root.children.items()]
            # print("here3")
            action, node2 = node2.select(self.cpuct, board2.possible_moves())
            # print (action)
            board2.pure_move(action)
        end = board2.is_game_over()
        if not end:
            action_probs2, leaf_value = self.policy(board2)
            node2.expand(action_probs2)
        else:
            winner = board2.winner()
            if winner == 0:  # tie
                leaf_value = 0.0
            else:
                leaf_value = (
                    1 if winner == board2.next_turn else -1
                )

        '''-ve because from the parent node's perspective the ,
     leaf node is -leaf_value as we use max while selecting the node.'''
        node2.update_ancestors(-leaf_value)

    def get_move_probs(self, board3, temp=0.8):

        for _ in range(self.n_playout):
            board_copy = copy.deepcopy(board3)
            self.playout(board_copy)

        act_visits = [(act, node.n_visits)
                      for act, node in self.root.children.items()]
        acts2, visits = zip(*act_visits)
        # print(acts2,visits)
        act_probs2 = np.array(visits) ** (1 / temp)
        # print(act_probs2)
        act_probs2 = act_probs2 / sum(act_probs2)
        # print(act_probs2)

        # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.root.children.items()]
        return acts2, act_probs2

    def update_with_move(self, move2):

        if move2 in self.root.children:
            temp=self.root.children[move2].children
            prob=self.root.children[move2].prior_prob
            self.root = Node(None, prob)
            self.children={}
            self.root.children=temp
            # print("-------------------------------------------------------------")
            # [print(act, f"{node.Q} {node.n_visits}") for act, node in self.root.children.items()]
            # print("-------------------------------------------------------------")
        else:
            self.root = Node(None, 1)

In [30]:
class HumanPlayer:
    def __init__(self):
        pass

    def get_action(self, board6):
        while True:
            move4 = input("Enter your move: ")
            try:
                board6.validate(move4)
                return move4
            except Exception as e:
                print(e)

class MCTSPlayer:

    def __init__(self, policy_value_fn,
                 cpuct=5, n_playout=2000, is_selfplay=0):
        self.mcts = MCTS(policy_value_fn, cpuct, n_playout)
        self.is_selfplay = is_selfplay

    def reset_player(self):
        self.mcts.update_with_move(-1)
        
    def update_move(self, move4):
        self.mcts.update_with_move(move4)
    
    def get_action(self, board6, temp=1e-3, return_prob=0):
        sensible_moves = board6.possible_moves()
        # print(sensible_moves)
        # the pi vector returned by MCTS as in the alphaGo Zero paper
        move_probs = np.zeros(217)
        if sensible_moves:
            acts, probs = self.mcts.get_move_probs(board6, temp)
            counter=0
            for act in acts:
                index=action_space[act]
                move_probs[index]=probs[counter]
                counter+=1

            if self.is_selfplay:
                # add Dirichlet Noise for exploration (needed for
                # self-play training)
                # print (acts, probs)
                move4 = np.random.choice(acts,p=
                0.75* probs + 0.25 *
                  np.random.dirichlet(0.3 * np.ones(len(probs))))
                # update the root node and reuse the search tree
                # print(move4)
                # print("here")
                self.mcts.update_with_move(move4)
            else:
                # with the default temp=1e-3, it is almost equivalent
                # to choosing the move with the highest prob
                # print (acts, probs)
                move4 = np.random.choice(acts, p=probs)
                # reset the root node
                self.mcts.update_with_move(move4)

            if return_prob:
                return move4, move_probs
            else:
                return move4
        else:
            print("WARNING ! Game is over.")


In [ ]:
#Self Play agents
class TrainPipeline():
    def __init__(self, init_model=None):

        # params of the board and the game
        self.game = Game()
        # training params
        self.config=TrainConfig()
        self.greedy_config=TrainGreedyConfig()
        self.play_config=PlayConfig()
        self.data_buffer = deque(maxlen=self.config.buffer_size)
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(init_model)
        else:
            # start training from a new policy-value net
            self.policy_value_net = PolicyValueNet()

        self.mcts_player = MCTSPlayer(self.policy_value_net.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout,
                                      is_selfplay=1)
        self.mcts_player_greedy = MCTSPlayer(self.policy_value_net.policy_value_fn,
                                      cpuct=self.greedy_config.c_puct,
                                      n_playout=self.greedy_config.n_playout,
                                      is_selfplay=1)

    def collect_selfplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Self-Play Data")
        print("Self-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            if os.path.exists(os.path.join(MODELPATH, 'best.h5')):
                self.policy_value_net=PolicyValueNet(os.path.join(MODELPATH, 'best.h5'))
                self.mcts_player=MCTSPlayer(self.policy_value_net.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout,
                                      is_selfplay=1)
            winner, play_data = self.game.start_self_play(self.mcts_player, show=False,
                                                          temp=self.config.temp)
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer.extend(play_data)
        print("\n Self-Play Data Collection Completed")

    def start_play_batch (self, GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=self.game.play_games(GoatPlayer, BaghPlayer, show=False, temp=self.play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count


    def policy_update(self, i):
        """update the policy-value net"""
        state_batch = [data[0] for data in self.data_buffer]
        mcts_probs_batch = [data[1] for data in self.data_buffer]
        winner_batch = [data[2] for data in self.data_buffer]
        self.policy_value_net.train(state_batch,mcts_probs_batch,winner_batch,self.config.epochs)
        if os.path.exists(os.path.join(MODELPATH, 'best.h5')):
            self.policy_value_net.save_model(os.path.join(MODELPATH, 'temp.h5'))
            newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp.h5'))
            newplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best.h5'))
            oldplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            print("Pitting new model vs previous best model")
            print("New player as goat")
            nwing, drawg, pwing =self.start_play_batch(newplayer, oldplayer, n_games=10, show=False)
            print([nwing, drawg, pwing])
            print("New player as bagh")
            pwinb, drawb, nwinb =self.start_play_batch(oldplayer, newplayer, n_games=10, show=False)
            print([nwinb, drawb, pwinb])
            print("Pitting Completed")
            nwin=nwing+nwinb
            pwin=pwing+pwinb
            draw=drawg+drawb
            print([nwin, draw, pwin])
            if (nwin>=pwin+1):
                print("New model accepted")
                self.policy_value_net.save_model(os.path.join(MODELPATH, 'best.h5'))
                modelname='accepted_'+str(i)+'.h5'
            else:
                print("New model rejected")
                modelname='rejected_'+str(i)+'.h5'
            self.policy_value_net.save_model(os.path.join(MODELPATH, modelname))
        else:
            self.policy_value_net.save_model(os.path.join(MODELPATH, 'best.h5'))
            self.policy_value_net.save_model(os.path.join(MODELPATH, 'accepted_0.h5'))

    def run(self):
        """run the training pipeline"""
        for i in range(100):
            try:
                self.collect_selfplay_data(self.config.play_batch_size)
                self.policy_update(i)
            except KeyboardInterrupt:
                print('\n\rquit')
    
    
    def summary(self):
        self.policy_value_net.model.summary()


if __name__ == '__main__':
    TrainPipeline().run()


In [ ]:
#Competitive agent with alternate training
class TrainPipelineCompetitive():
    def __init__(self, init_model=None):

        # params of the board and the game
        self.game = Game()
        self.turn = "G"
        # training params
        self.config=TrainConfig()
        self.greedy_config=TrainGreedyConfig()
        self.play_config=PlayConfig()
        self.data_buffer = deque(maxlen=self.config.buffer_size)
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(init_model)
        else:
            # start training from a new policy-value net
            self.policy_value_net_goat = PolicyValueNet()
            self.policy_value_net_bagh = PolicyValueNet()

        self.mcts_player_goat = MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        self.mcts_player_bagh = MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        # self.mcts_player_greedy = MCTSPlayer(self.policy_value_net.policy_value_fn,
        #                               cpuct=self.greedy_config.c_puct,
        #                               n_playout=self.greedy_config.n_playout,
        #                               is_selfplay=1)

    def collect_compplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Comp-Play Data")
        print("Comp-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                self.mcts_player_goat=MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.mcts_player_bagh=MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            winner, play_data = self.game.play_games(self.mcts_player_goat, self.mcts_player_bagh, show=False, temp=self.config.temp)
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer.extend(play_data)
        print("\n Comp-Play Data Collection Completed")

    def start_play_batch_competitve (self, GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=self.game.play_games(GoatPlayer, BaghPlayer, show=False, temp=self.play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count


    def policy_update_competitive(self, i):
        """update the policy-value net"""
        state_batch = [data[0] for data in self.data_buffer]
        mcts_probs_batch = [data[1] for data in self.data_buffer]
        winner_batch = [data[2] for data in self.data_buffer]
        if self.turn=="G":
            self.policy_value_net_goat.train(state_batch,mcts_probs_batch,winner_batch,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'temp_goat.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_goat.h5'))
                newgoatplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                oldbaghplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new goat model vs previous bagh model")
                nwin, draw, pwin =self.start_play_batch_competitve(newgoatplayer, oldbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin>=pwin+1):
                    print("New goat model accepted")
                    self.turn="B"
                    self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                    modelname='acceptedgoat_'+str(i)+'.h5'
                else:
                    print("New goat model rejected")
                    modelname='rejectedgoat_'+str(i)+'.h5'
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="B"
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'acceptedgoat_0.h5'))
        else:
            winner_batch2 = [data*-1 for data in winner_batch]
            self.policy_value_net_bagh.train(state_batch,mcts_probs_batch,winner_batch2,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newbaghplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                oldgoatplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new bagh model vs previous goat model")
                pwin, draw, nwin =self.start_play_batch_competitve(oldgoatplayer, newbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin>=pwin+1):
                    print("New bagh model accepted")
                    self.turn="G"
                    self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                    modelname='acceptedbagh_'+str(i)+'.h5'
                else:
                    print("New bagh model rejected")
                    modelname='rejectedbagh_'+str(i)+'.h5'
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="G"
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'acceptedbagh_0.h5'))

    def run(self):
        """run the training pipeline"""
        for i in range(200):
            try:
                self.collect_compplay_data(self.config.play_batch_size)
                self.policy_update_competitive(i)
            except KeyboardInterrupt:
                print('\n\rquit')
    
    
    def summary(self):
        self.policy_value_net_goat.model.summary()
        self.policy_value_net_bagh.model.summary()


if __name__ == '__main__':
    TrainPipelineCompetitive().run()


In [ ]:
#Competitive agents with data filter
class TrainPipelineCompetitive():
    def __init__(self, init_model=None):

        # params of the board and the game
        self.game = Game()
        self.turn = "G"
        # training params
        self.config=TrainConfig()
        self.greedy_config=TrainGreedyConfig()
        self.play_config=PlayConfig()
        self.data_buffer_goat = deque(maxlen=self.config.buffer_size)
        self.data_buffer_bagh = deque(maxlen=self.config.buffer_size)
        self.oldgoat=-20
        self.oldbagh=-20
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(init_model)
        else:
            # start training from a new policy-value net
            self.policy_value_net_goat = PolicyValueNet()
            self.policy_value_net_bagh = PolicyValueNet()

        self.mcts_player_goat = MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        self.mcts_player_bagh = MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        # self.mcts_player_greedy = MCTSPlayer(self.policy_value_net.policy_value_fn,
        #                               cpuct=self.greedy_config.c_puct,
        #                               n_playout=self.greedy_config.n_playout,
        #                               is_selfplay=1)

    def collect_compplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Comp-Play Data")
        print("Comp-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                self.mcts_player_goat=MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.mcts_player_bagh=MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            winner, play_data = self.game.play_games(self.mcts_player_goat, self.mcts_player_bagh, show=False, temp=self.config.temp)
            print(winner, end=" ")
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            if winner=="G":
                for data in play_data:
                    if data[2]==1:
                        self.data_buffer_goat.append(data)
                    else:
                        self.data_buffer_bagh.append(data)
            if winner=="B":
                  for data in play_data:
                    if data[2]==1:
                        self.data_buffer_bagh.append(data)
                    else:
                        self.data_buffer_goat.append(data)
            if winner==0:
                self.data_buffer_goat.extend(play_data)
                self.data_buffer_bagh.extend(play_data)
        print("\n Comp-Play Data Collection Completed")

    def start_play_batch_competitve (self, GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=self.game.play_games(GoatPlayer, BaghPlayer, show=False, temp=self.play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count


    def policy_update_competitive(self, i):
        """update the policy-value net"""
        state_batch_goat = [data[0] for data in self.data_buffer_goat]
        mcts_probs_batch_goat = [data[1] for data in self.data_buffer_goat]
        winner_batch_goat = [data[2] for data in self.data_buffer_goat]
        print(winner_batch_goat)
        state_batch_bagh = [data[0] for data in self.data_buffer_bagh]
        mcts_probs_batch_bagh = [data[1] for data in self.data_buffer_bagh]
        winner_batch_bagh = [data[2] for data in self.data_buffer_bagh]
        print(winner_batch_bagh)
        if self.turn=="G":
            self.policy_value_net_goat.train(state_batch_goat,mcts_probs_batch_goat,winner_batch_goat,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'temp_goat.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_goat.h5'))
                newgoatplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                oldbaghplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new goat model vs previous bagh model")
                nwin, draw, pwin =self.start_play_batch_competitve(newgoatplayer, oldbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin-pwin>self.oldgoat):
                    print("New goat model accepted")
                    self.turn="B"
                    self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                    modelname='acceptedgoat_'+str(i)+'.h5'
                    self.oldbagh=pwin-nwin
                else:
                    print("New goat model rejected")
                    modelname='rejectedgoat_'+str(i)+'.h5'
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="B"
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'acceptedgoat_0.h5'))
        else:
            # winner_batch2 = [data*-1 for data in winner_batch]
            self.policy_value_net_bagh.train(state_batch_bagh,mcts_probs_batch_bagh,winner_batch_bagh,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newbaghplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                oldgoatplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new bagh model vs previous goat model")
                pwin, draw, nwin =self.start_play_batch_competitve(oldgoatplayer, newbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin-pwin>self.oldbagh):
                    print("New bagh model accepted")
                    self.turn="G"
                    self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                    modelname='acceptedbagh_'+str(i)+'.h5'
                    self.oldgoat=pwin-nwin
                else:
                    print("New bagh model rejected")
                    modelname='rejectedbagh_'+str(i)+'.h5'
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="G"
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'acceptedbagh_0.h5'))

    def run(self):
        """run the training pipeline"""
        for i in range(150):
            try:
                self.collect_compplay_data(self.config.play_batch_size)
                self.policy_update_competitive(i)
            except KeyboardInterrupt:
                print('\n\rquit')
    
    
    def summary(self):
        self.policy_value_net_goat.model.summary()
        self.policy_value_net_bagh.model.summary()


if __name__ == '__main__':
    TrainPipelineCompetitive().run()


In [ ]:
#Competitive agent with accepted condition changed
class TrainPipelineCompetitive():
    def __init__(self, init_model=None):

        # params of the board and the game
        self.game = Game()
        self.turn = "G"
        # training params
        self.config=TrainConfig()
        self.greedy_config=TrainGreedyConfig()
        self.play_config=PlayConfig()
        self.data_buffer = deque(maxlen=self.config.buffer_size)
        self.oldgoat=-20
        self.oldbagh=-20
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(init_model)
        else:
            # start training from a new policy-value net
            self.policy_value_net_goat = PolicyValueNet()
            self.policy_value_net_bagh = PolicyValueNet()

        self.mcts_player_goat = MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        self.mcts_player_bagh = MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        # self.mcts_player_greedy = MCTSPlayer(self.policy_value_net.policy_value_fn,
        #                               cpuct=self.greedy_config.c_puct,
        #                               n_playout=self.greedy_config.n_playout,
        #                               is_selfplay=1)

    def collect_compplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Comp-Play Data")
        print("Comp-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                self.mcts_player_goat=MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.mcts_player_bagh=MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            winner, play_data = self.game.play_games(self.mcts_player_goat, self.mcts_player_bagh, show=False, temp=self.config.temp)
            print(winner, end=" ")
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer.extend(play_data)
        print("\n Comp-Play Data Collection Completed")

    def start_play_batch_competitve (self, GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=self.game.play_games(GoatPlayer, BaghPlayer, show=False, temp=self.play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count


    def policy_update_competitive(self, i):
        """update the policy-value net"""
        state_batch_goat = [data[0] for data in self.data_buffer]
        mcts_probs_batch_goat = [data[1] for data in self.data_buffer]
        winner_batch_goat = [data[2] for data in self.data_buffer]
        print(winner_batch_goat)
        state_batch_bagh = [data[0] for data in self.data_buffer]
        mcts_probs_batch_bagh = [data[1] for data in self.data_buffer]
        winner_batch_bagh = [data[2] for data in self.data_buffer]
        print(winner_batch_bagh)
        if self.turn=="G":
            self.policy_value_net_goat.train(state_batch_goat,mcts_probs_batch_goat,winner_batch_goat,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'temp_goat.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_goat.h5'))
                newgoatplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                oldbaghplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new goat model vs previous bagh model")
                nwin, draw, pwin =self.start_play_batch_competitve(newgoatplayer, oldbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin-pwin>self.oldgoat):
                    print("New goat model accepted")
                    self.turn="B"
                    self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                    modelname='acceptedgoat_'+str(i)+'.h5'
                    self.oldbagh=pwin-nwin
                else:
                    print("New goat model rejected")
                    modelname='rejectedgoat_'+str(i)+'.h5'
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="B"
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'acceptedgoat_0.h5'))
        else:
            # winner_batch2 = [data*-1 for data in winner_batch]
            self.policy_value_net_bagh.train(state_batch_bagh,mcts_probs_batch_bagh,winner_batch_bagh,self.config.epochs)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_bagh.h5'))
                newbaghplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                oldgoatplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
                print("Pitting new bagh model vs previous goat model")
                pwin, draw, nwin =self.start_play_batch_competitve(oldgoatplayer, newbaghplayer, n_games=10, show=False)
                print([nwin, draw, pwin])
                if (nwin-pwin>self.oldbagh):
                    print("New bagh model accepted")
                    self.turn="G"
                    self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                    modelname='acceptedbagh_'+str(i)+'.h5'
                    self.oldgoat=pwin-nwin
                else:
                    print("New bagh model rejected")
                    modelname='rejectedbagh_'+str(i)+'.h5'
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, modelname))
            else:
                self.turn="G"
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'acceptedbagh_0.h5'))

    def run(self):
        """run the training pipeline"""
        for i in range(150):
            try:
                self.collect_compplay_data(self.config.play_batch_size)
                self.policy_update_competitive(i)
            except KeyboardInterrupt:
                print('\n\rquit')
    
    
    def summary(self):
        self.policy_value_net_goat.model.summary()
        self.policy_value_net_bagh.model.summary()


if __name__ == '__main__':
    TrainPipelineCompetitive().run()


In [ ]:
#Compeititve agents with human play data injection
class TrainPipelineCompetitive():
    def __init__(self, init_model=None):

        # params of the board and the game
        self.game = Game()
        self.turn = "G"
        # training params
        self.config=TrainConfig()
        self.greedy_config=TrainGreedyConfig()
        self.play_config=PlayConfig()
        self.data_buffer = deque(maxlen=self.config.buffer_size)
        self.data_buffer_human  = deque(maxlen=self.config.buffer_size)
        self.oldgoat=-1
        self.oldbagh=-1
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(init_model)
        else:
            # start training from a new policy-value net
            self.policy_value_net_goat = PolicyValueNet()
            self.policy_value_net_bagh = PolicyValueNet()

        self.mcts_player_goat = MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        self.mcts_player_bagh = MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
        # self.mcts_player_greedy = MCTSPlayer(self.policy_value_net.policy_value_fn,
        #                               cpuct=self.greedy_config.c_puct,
        #                               n_playout=self.greedy_config.n_playout,
        #                               is_selfplay=1)


    

    def collect_humanplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Human-Play Data")
        print("Human-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            humplayer=HumanPlayer()
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                self.mcts_player_goat=MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.play_config.c_puct,
                                      n_playout=self.play_config.n_playout)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.mcts_player_bagh=MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.play_config.c_puct,
                                      n_playout=self.play_config.n_playout)
            winner, play_data = self.game.start_play(humplayer, self.mcts_player_bagh)
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer_human.extend(play_data)

            winner, play_data = self.game.start_play(self.mcts_player_goat, humplayer)
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer_human.extend(play_data)

        print("\n Human-Play Data Collection Completed")


    def collect_compplay_data(self, n_games=1):
        """collect self-play data for training"""
        print("Collecting Comp-Play Data")
        print("Comp-Play Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            # winner, play_data = self.game.start_self_play(self.mcts_player,
            #                                               temp=self.config.temp
            #                                               ,greedy_player=self.mcts_player_greedy,who_greedy="B")
            if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
                self.policy_value_net_goat=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
                self.mcts_player_goat=MCTSPlayer(self.policy_value_net_goat.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
                self.policy_value_net_bagh=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
                self.mcts_player_bagh=MCTSPlayer(self.policy_value_net_bagh.policy_value_fn,
                                      cpuct=self.config.c_puct,
                                      n_playout=self.config.n_playout)
            winner, play_data = self.game.play_games(self.mcts_player_goat, self.mcts_player_bagh, show=False, temp=self.config.temp)
            print(winner, end=" ")
            play_data = list(play_data)
            # augment the data
            play_data = symmetry_board_moves(play_data)
            self.data_buffer.extend(play_data)
        print("\n Comp-Play Data Collection Completed")

    def start_play_batch_competitve (self, GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=self.game.play_games(GoatPlayer, BaghPlayer, show=False, temp=self.play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count


    def policy_update_competitive(self, i):
        """update the policy-value net"""
        self.saveTrainExamples()
        self.saveHumanExamples()
        # if i%10==0 and i!=0:
        if i%10==0:
            self.saveHumanExamples()
        
        state_batch = [data[0] for data in self.data_buffer]
        mcts_probs_batch = [data[1] for data in self.data_buffer]
        winner_batch = [data[2] for data in self.data_buffer]
        
        # if i>=10:
        if i>=0:
            state_batch.extend([data[0] for data in self.data_buffer_human])
            mcts_probs_batch.extend([data[1] for data in self.data_buffer_human])
            winner_batch.extend([data[2] for data in self.data_buffer_human])
        self.policy_value_net_goat.train(state_batch,mcts_probs_batch,winner_batch,self.config.epochs)
        if os.path.exists(os.path.join(MODELPATH, 'best_goat.h5')):
            self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'temp_goat.h5'))
            newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_goat.h5'))
            newgoatplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_bagh.h5'))
            oldbaghplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            print("Pitting new goat model vs previous bagh model")
            nwin, draw, pwin =self.start_play_batch_competitve(newgoatplayer, oldbaghplayer, n_games=10, show=False)
            print([nwin, draw, pwin])
            if (nwin-pwin>self.oldgoat):
                print("New goat model accepted")
                self.turn="B"
                self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
                modelname='acceptedgoat_'+str(i)+'.h5'
                self.oldbagh=pwin-nwin
            else:
                print("New goat model rejected")
                modelname='rejectedgoat_'+str(i)+'.h5'
            self.policy_value_net_goat.save_model(os.path.join(MODELPATH, modelname))
        else:
            self.turn="B"
            self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'best_goat.h5'))
            self.policy_value_net_goat.save_model(os.path.join(MODELPATH, 'acceptedgoat_0.h5'))
        # winner_batch2 = [data*-1 for data in winner_batch]
        self.policy_value_net_bagh.train(state_batch,mcts_probs_batch,winner_batch,self.config.epochs)
        if os.path.exists(os.path.join(MODELPATH, 'best_bagh.h5')):
            self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'temp_bagh.h5'))
            newnet=PolicyValueNet(os.path.join(MODELPATH, 'temp_bagh.h5'))
            newbaghplayer=MCTSPlayer(newnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            oldnet=PolicyValueNet(os.path.join(MODELPATH, 'best_goat.h5'))
            oldgoatplayer=MCTSPlayer(oldnet.policy_value_fn, cpuct=self.play_config.c_puct, n_playout=self.play_config.n_playout)
            print("Pitting new bagh model vs previous goat model")
            pwin, draw, nwin =self.start_play_batch_competitve(oldgoatplayer, newbaghplayer, n_games=10, show=False)
            print([nwin, draw, pwin])
            if (nwin-pwin>self.oldbagh):
                print("New bagh model accepted")
                self.turn="G"
                self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
                modelname='acceptedbagh_'+str(i)+'.h5'
                self.oldgoat=pwin-nwin
            else:
                print("New bagh model rejected")
                modelname='rejectedbagh_'+str(i)+'.h5'
            self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, modelname))
        else:
            self.turn="G"
            self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'best_bagh.h5'))
            self.policy_value_net_bagh.save_model(os.path.join(MODELPATH, 'acceptedbagh_0.h5'))

    def saveTrainExamples(self):
        filename = os.path.join(MODELPATH, "complaydata.examples")
        with open(filename, "wb+") as f:
            Pickler(f).dump(self.data_buffer)
        f.closed
        

    def saveHumanExamples(self):
        filename = os.path.join(MODELPATH, "humplaydata.examples")
        with open(filename, "wb+") as f:
            Pickler(f).dump(self.data_buffer_human)
        f.closed

    def loadTrainExamples(self):
        examplesFile = os.path.join(MODELPATH, "complaydata.examples")
        if os.path.isfile(examplesFile):
            with open(examplesFile, "rb") as f:
                self.data_buffer = Unpickler(f).load()

    def loadHumanExamples(self):
        examplesFile = os.path.join(MODELPATH, "humplaydata.examples")
        if os.path.isfile(examplesFile):
            with open(examplesFile, "rb") as f:
                self.data_buffer_human = Unpickler(f).load()
            
    
    def run(self):
        """run the training pipeline"""
        self.loadTrainExamples()
        self.loadHumanExamples()

        # if os.path.exists(os.path.join(MODELPATH, 'humanplaydata')):
        #     file = open(os.path.join(MODELPATH, 'humanplaydata'))
        #     csvreader = csv.reader(file)
        #     rows = []
        #     for row in csvreader:
        #         rows.append(row)
        #     file.close()
        #     self.data_buffer_human=rows[0]

        for i in range(150):
            try:
                # if i%10==0 and i!=0:
                if i%10==0:
                    self.collect_humanplay_data(2)
                self.collect_compplay_data(self.config.play_batch_size)
                self.policy_update_competitive(i+21)
            except KeyboardInterrupt:
                print('\n\rquit')
    
    
    def summary(self):
        self.policy_value_net_goat.model.summary()
        self.policy_value_net_bagh.model.summary()

if __name__ == '__main__':
    TrainPipelineCompetitive().run()


In [ ]:
#Pitting Self play agents with first accepted agents
game = Game()
play_config=PlayConfig()
# all players
basenet=PolicyValueNet(os.path.join(MODELPATH, 'accepted_0.h5'))
baseplayer=MCTSPlayer(basenet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)
nwinlist=[]
pwinlist=[]
drawlist=[]
adjustedwin=[]
def start_play_batch (GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=game.play_games(GoatPlayer, BaghPlayer, show=False, temp=play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count

for i in range(100):
    filename='accepted_'+str(i)+'.h5'
    print(filename)
    if os.path.exists(os.path.join(MODELPATH, filename)):
        acceptednet=PolicyValueNet(os.path.join(MODELPATH, filename))
        acceptedplayer=MCTSPlayer(acceptednet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

        player2 = baseplayer  # Player 2 is neural network if it's cpu vs cpu.

        print("Accepted player as goat")
        nwing, drawg, pwing =start_play_batch(acceptedplayer, baseplayer , n_games=10, show=False)
        print([nwing, drawg, pwing])
        print("Accepted player as bagh")
        pwinb, drawb, nwinb =start_play_batch(baseplayer, acceptedplayer, n_games=10, show=False)
        print([nwinb, drawb, pwinb])
        print("Pitting Completed")
        nwin=nwing+nwinb
        pwin=pwing+pwinb
        draw=drawg+drawb
        nwinlist.append(nwin/20)
        pwinlist.append(pwin/20)
        drawlist.append(draw/20)
        adjustedwin.append((nwin)/(20-draw)
        print([nwin, draw, pwin])
        
        f = open(os.path.join(RESULTPATH, 'selfresults'), 'a')
        writer = csv.writer(f)
        tempscore=[nwin, draw, pwin]
        writer.writerow(tempscore)
        f.close()
    else:
        print('file not found')

plt.plot(range(1,len(nwinlist)+1),nwinlist, label='Accepted agent win probability')
plt.plot(range(1,len(pwinlist)+1),pwinlist, label='Accepted agent loss probability')
plt.plot(range(1,len(drawlist)+1),drawlist, label='Accepted agent draw probability')
plt.legend()
plt.title('Accepted agents vs the first accepted agent')
plt.ylabel('Probability')
plt.xlabel('Accepted Agents')
plt.show()

plt.plot(range(1,len(adjustedwin)+1),adjustedwin, label='Accepted agent adjusted win probability')
plt.legend()
plt.title('Accepted agents vs the first accepted agent (adjusted win rate)')
plt.ylabel('Probability')
plt.xlabel('Accepted Agents')
plt.show()

In [ ]:
#Pitting AT agents with self_play agents
game = Game()
play_config=PlayConfig()
# all players
bestagents=['accepted_0.h5',
            'accepted_61.h5',
            'accepted_58.h5',
            'accepted_55.h5',
            'accepted_45.h5',
            'accepted_32.h5']
            
goatnet=PolicyValueNet(os.path.join(COMPPATH, 'best_goat.h5'))
goatplayer=MCTSPlayer(goatnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

baghnet=PolicyValueNet(os.path.join(COMPPATH, 'best_bagh.h5'))
baghplayer=MCTSPlayer(baghnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

def start_play_batch (GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=game.play_games(GoatPlayer, BaghPlayer, show=False, temp=play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count

for agent in bestagents:
    print(agent)
    if os.path.exists(os.path.join(MODELPATH, agent)):
        acceptednet=PolicyValueNet(os.path.join(MODELPATH, agent))
        acceptedplayer=MCTSPlayer(acceptednet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

        print("Goat Player")
        nwing, drawg, pwing =start_play_batch(goatplayer, acceptedplayer , n_games=20, show=False)
        print([nwing, drawg, pwing])

        f = open(os.path.join(RESULTPATH, 'compresultsgoat'), 'a')
        writer = csv.writer(f)
        tempscore=[nwing, drawg, pwing]
        writer.writerow(tempscore)
        f.close()

        print("Bagh Player")
        pwinb, drawb, nwinb =start_play_batch(acceptedplayer, baghplayer, n_games=20, show=False)
        print([nwinb, drawb, pwinb])
        
        f = open(os.path.join(RESULTPATH, 'compresultsbagh'), 'a')
        writer = csv.writer(f)
        tempscore=[nwinb, drawb, pwinb]
        writer.writerow(tempscore)
        f.close()
    else:
        print('file not found')

In [ ]:
#Pitting Acceptance condition changed agents with self play agents
game = Game()
play_config=PlayConfig()
# all players
bestagents=['accepted_0.h5',
            'accepted_61.h5',
            'accepted_58.h5',
            'accepted_55.h5',
            'accepted_45.h5',
            'accepted_32.h5']
            
goatnet=PolicyValueNet(os.path.join(ACCEPTPATH, 'best_goat.h5'))
goatplayer=MCTSPlayer(goatnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

baghnet=PolicyValueNet(os.path.join(ACCEPTPATH, 'best_bagh.h5'))
baghplayer=MCTSPlayer(baghnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

def start_play_batch (GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=game.play_games(GoatPlayer, BaghPlayer, show=False, temp=play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count

for agent in bestagents:
    print(agent)
    if os.path.exists(os.path.join(MODELPATH, agent)):
        acceptednet=PolicyValueNet(os.path.join(MODELPATH, agent))
        acceptedplayer=MCTSPlayer(acceptednet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

        print("Goat Player")
        nwing, drawg, pwing =start_play_batch(goatplayer, acceptedplayer , n_games=20, show=False)
        print([nwing, drawg, pwing])

        f = open(os.path.join(RESULTPATH, 'acceptresultsgoat'), 'a')
        writer = csv.writer(f)
        tempscore=[nwing, drawg, pwing]
        writer.writerow(tempscore)
        f.close()

        print("Bagh Player")
        pwinb, drawb, nwinb =start_play_batch(acceptedplayer, baghplayer, n_games=20, show=False)
        print([nwinb, drawb, pwinb])
        
        f = open(os.path.join(RESULTPATH, 'acceptresultsbagh'), 'a')
        writer = csv.writer(f)
        tempscore=[nwinb, drawb, pwinb]
        writer.writerow(tempscore)
        f.close()
    else:
        print('file not found')

In [ ]:
#Pitting Data filter agents with self play agents
game = Game()
play_config=PlayConfig()
# all players
bestagents=['accepted_0.h5',
            'accepted_61.h5',
            'accepted_58.h5',
            'accepted_55.h5',
            'accepted_45.h5',
            'accepted_32.h5']
            
goatnet=PolicyValueNet(os.path.join(FILTERPATH, 'best_goat.h5'))
goatplayer=MCTSPlayer(goatnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

baghnet=PolicyValueNet(os.path.join(FILTERPATH, 'best_bagh.h5'))
baghplayer=MCTSPlayer(baghnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

def start_play_batch (GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=game.play_games(GoatPlayer, BaghPlayer, show=False, temp=play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count

for agent in bestagents:
    print(agent)
    if os.path.exists(os.path.join(MODELPATH, agent)):
        acceptednet=PolicyValueNet(os.path.join(MODELPATH, agent))
        acceptedplayer=MCTSPlayer(acceptednet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

        print("Goat Player")
        nwing, drawg, pwing =start_play_batch(goatplayer, acceptedplayer , n_games=20, show=False)
        print([nwing, drawg, pwing])

        f = open(os.path.join(RESULTPATH, 'filterresultsgoat'), 'a')
        writer = csv.writer(f)
        tempscore=[nwing, drawg, pwing]
        writer.writerow(tempscore)
        f.close()

        print("Bagh Player")
        pwinb, drawb, nwinb =start_play_batch(acceptedplayer, baghplayer, n_games=20, show=False)
        print([nwinb, drawb, pwinb])
        
        f = open(os.path.join(RESULTPATH, 'filterresultsbagh'), 'a')
        writer = csv.writer(f)
        tempscore=[nwinb, drawb, pwinb]
        writer.writerow(tempscore)
        f.close()
    else:
        print('file not found')

In [ ]:
#Pitting human play agents with self play agents
game = Game()
play_config=PlayConfig()
# all players
bestagents=['accepted_0.h5',
            'accepted_61.h5',
            'accepted_58.h5',
            'accepted_55.h5',
            'accepted_45.h5',
            'accepted_32.h5']
            
goatnet=PolicyValueNet(os.path.join(HUMANPATH, 'best_goat_19.h5'))
goatplayer=MCTSPlayer(goatnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

baghnet=PolicyValueNet(os.path.join(HUMANPATH, 'best_bagh.h5'))
baghplayer=MCTSPlayer(baghnet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

def start_play_batch (GoatPlayer, BaghPlayer, n_games=1, show=True):
        goat_count=0
        draw_count=0
        bagh_count=0
        print("Pitting Games", end =" ")
        for i in range(n_games):
            print(str(i), end =" ")
            winner, data=game.play_games(GoatPlayer, BaghPlayer, show=False, temp=play_config.temp)
            print(winner, end =" ")
            if (winner=="B"):
                bagh_count+=1
            elif (winner=="G"):
                goat_count+=1
            else:
                draw_count+=1
        return goat_count, draw_count, bagh_count

for agent in bestagents:
    print(agent)
    if os.path.exists(os.path.join(MODELPATH, agent)):
        acceptednet=PolicyValueNet(os.path.join(MODELPATH, agent))
        acceptedplayer=MCTSPlayer(acceptednet.policy_value_fn, cpuct=play_config.c_puct, n_playout=play_config.n_playout)

        print("Goat Player")
        nwing, drawg, pwing =start_play_batch(goatplayer, acceptedplayer , n_games=20, show=False)
        print([nwing, drawg, pwing])

        f = open(os.path.join(RESULTPATH, 'humanresultsgoat'), 'a')
        writer = csv.writer(f)
        tempscore=[nwing, drawg, pwing]
        writer.writerow(tempscore)
        f.close()

        print("Bagh Player")
        pwinb, drawb, nwinb =start_play_batch(acceptedplayer, baghplayer, n_games=20, show=False)
        print([nwinb, drawb, pwinb])
        
        f = open(os.path.join(RESULTPATH, 'humanresultsbagh'), 'a')
        writer = csv.writer(f)
        tempscore=[nwinb, drawb, pwinb]
        writer.writerow(tempscore)
        f.close()
    else:
        print('file not found')

In [ ]:
#Generating graphs from csv
file = open(os.path.join(RESULTPATH, 'selfresults'))
csvreader = csv.reader(file)
rows = []
for row in csvreader:
    rows.append(row)
file.close()

file2 = open(os.path.join(RESULTPATH, 'compresultsgoat'))
csvreader2 = csv.reader(file2)
rows2 = []
for row in csvreader2:
    rows2.append(row)
file2.close()

file3 = open(os.path.join(RESULTPATH, 'compresultsbagh'))
csvreader3 = csv.reader(file3)
rows3 = []
for row in csvreader3:
    rows3.append(row)
file3.close()

file4 = open(os.path.join(RESULTPATH, 'acceptresultsgoat'))
csvreader4 = csv.reader(file4)
rows4 = []
for row in csvreader4:
    rows4.append(row)
file4.close()

file5 = open(os.path.join(RESULTPATH, 'acceptresultsbagh'))
csvreader5 = csv.reader(file5)
rows5 = []
for row in csvreader5:
    rows5.append(row)
file5.close()

file6 = open(os.path.join(RESULTPATH, 'filterresultsgoat'))
csvreader6 = csv.reader(file6)
rows6 = []
for row in csvreader6:
    rows6.append(row)
file6.close()

file7 = open(os.path.join(RESULTPATH, 'filterresultsbagh'))
csvreader7 = csv.reader(file7)
rows7 = []
for row in csvreader7:
    rows7.append(row)
file7.close()

file8 = open(os.path.join(RESULTPATH, 'humanresultsgoat'))
csvreader8 = csv.reader(file8)
rows8 = []
for row in csvreader8:
    rows8.append(row)
file8.close()

file9 = open(os.path.join(RESULTPATH, 'humanresultsbagh'))
csvreader9 = csv.reader(file9)
rows9 = []
for row in csvreader9:
    rows9.append(row)
file9.close()

nwin=[int(row[0]) for row in rows]
draw=[int(row[1]) for row in rows]
pwin=[int(row[2]) for row in rows]
nwinlist=[int(row[0])/20 for row in rows]
pwinlist=[int(row[2])/20 for row in rows]
drawlist=[int(row[1])/20 for row in rows]

nwincompgoat=[int(row[0]) for row in rows2]
drawcompgoat=[int(row[1]) for row in rows2]
pwincompgoat=[int(row[2]) for row in rows2]
nwinlistcompgoat=[int(row[0])/20 for row in rows2]
pwinlistcompgoat=[int(row[2])/20 for row in rows2]
drawlistcompgoat=[int(row[1])/20 for row in rows2]

nwincompbagh=[int(row[0]) for row in rows3]
drawcompbagh=[int(row[1]) for row in rows3]
pwincompbagh=[int(row[2]) for row in rows3]
nwinlistcompbagh=[int(row[0])/20 for row in rows3]
pwinlistcompbagh=[int(row[2])/20 for row in rows3]
drawlistcompbagh=[int(row[1])/20 for row in rows3]


nwinlistacceptgoat=[int(row[0])/20 for row in rows4]
pwinlistacceptgoat=[int(row[2])/20 for row in rows4]
drawlistacceptgoat=[int(row[1])/20 for row in rows4]

nwinlistacceptbagh=[int(row[0])/20 for row in rows5]
pwinlistacceptbagh=[int(row[2])/20 for row in rows5]
drawlistacceptbagh=[int(row[1])/20 for row in rows5]

nwinlistfiltergoat=[int(row[0])/20 for row in rows6]
pwinlistfiltergoat=[int(row[2])/20 for row in rows6]
drawlistfiltergoat=[int(row[1])/20 for row in rows6]

nwinlistfilterbagh=[int(row[0])/20 for row in rows7]
pwinlistfilterbagh=[int(row[2])/20 for row in rows7]
drawlistfilterbagh=[int(row[1])/20 for row in rows7]

nwinlisthumangoat=[int(row[0])/20 for row in rows8]
pwinlisthumangoat=[int(row[2])/20 for row in rows8]
drawlisthumangoat=[int(row[1])/20 for row in rows8]

nwinlisthumanbagh=[int(row[0])/20 for row in rows9]
pwinlisthumanbagh=[int(row[2])/20 for row in rows9]
drawlisthumanbagh=[int(row[1])/20 for row in rows9]

adjustedwin=[(int(row[0]))/(20-int(row[2])) for row in rows]

#graphs for self play agents
plt.plot(range(0,len(nwinlist)),nwinlist, label='Accepted agent win rate')
plt.plot(range(0,len(pwinlist)),pwinlist, label='Accepted agent lose rate')
plt.plot(range(0,len(drawlist)),drawlist, label='Accepted agent draw rate')
plt.legend(loc='upper left')
plt.title('Accepted agents vs the first accepted agent (base)')
plt.ylabel('Probability')
plt.xlabel('Accepted self play agents')
plt.show()

plt.plot(range(0,len(nwin)),nwin, label='Accepted agent wins')
plt.plot(range(0,len(pwin)),pwin, label='Accepted agent losses')
plt.plot(range(0,len(pwin)),draw, label='Accepted agent draws')
plt.legend(loc='upper left')
plt.title('Accepted agents vs the first accepted agent (count)')
plt.ylabel('Count')
plt.xlabel('Accepted Agents')
plt.show()

#graphs for alternate training agents
plt.plot(range(0,len(nwinlistcompgoat)),nwinlistcompgoat, label='Comp Goat Win rate')
plt.plot(range(0,len(pwinlistcompgoat)),pwinlistcompgoat, label='Comp Goat lose rate')
plt.plot(range(0,len(drawlistcompgoat)),drawlistcompgoat, label='Comp Goat draw rate')
plt.legend(loc='upper left')
plt.title('Best AT goat vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

plt.plot(range(0,len(nwinlistcompbagh)),nwinlistcompbagh, label='Comp Bagh win rate')
plt.plot(range(0,len(pwinlistcompbagh)),pwinlistcompbagh, label='Comp Bagh lose rate')
plt.plot(range(0,len(drawlistcompbagh)),drawlistcompbagh, label='Comp Bagh draw rate')
plt.legend(loc='upper left')
plt.title('Best AT tiger vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

#graphs for accepted condition changed agents
plt.plot(range(0,len(nwinlistacceptgoat)),nwinlistacceptgoat, label='Comp Goat Win rate')
plt.plot(range(0,len(pwinlistacceptgoat)),pwinlistacceptgoat, label='Comp Goat lose rate')
plt.plot(range(0,len(drawlistacceptgoat)),drawlistacceptgoat, label='Comp Goat draw rate')
plt.legend(loc='upper left')
plt.title('Best ACC goat vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

plt.plot(range(0,len(nwinlistacceptbagh)),nwinlistacceptbagh, label='Comp Bagh win rate')
plt.plot(range(0,len(pwinlistacceptbagh)),pwinlistacceptbagh, label='Comp Bagh lose rate')
plt.plot(range(0,len(drawlistacceptbagh)),drawlistacceptbagh, label='Comp Bagh draw rate')
plt.legend(loc='upper left')
plt.title('Best ACC tiger vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

#graphs for data filter agents
plt.plot(range(0,len(nwinlistfiltergoat)),nwinlistfiltergoat, label='Comp Goat Win rate')
plt.plot(range(0,len(pwinlistfiltergoat)),pwinlistfiltergoat, label='Comp Goat lose rate')
plt.plot(range(0,len(drawlistfiltergoat)),drawlistfiltergoat, label='Comp Goat draw rate')
plt.legend(loc='upper left')
plt.title('Best DF goat vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

plt.plot(range(0,len(nwinlistfilterbagh)),nwinlistfilterbagh, label='Comp Bagh win rate')
plt.plot(range(0,len(pwinlistfilterbagh)),pwinlistfilterbagh, label='Comp Bagh lose rate')
plt.plot(range(0,len(drawlistfilterbagh)),drawlistfilterbagh, label='Comp Bagh draw rate')
plt.legend(loc='upper left')
plt.title('Best DF tiger vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

#graphs for human play agents
plt.plot(range(0,len(nwinlisthumangoat)),nwinlisthumangoat, label='Comp Goat Win rate')
plt.plot(range(0,len(pwinlisthumangoat)),pwinlisthumangoat, label='Comp Goat lose rate')
plt.plot(range(0,len(drawlisthumangoat)),drawlisthumangoat, label='Comp Goat draw rate')
plt.legend(loc='upper left')
plt.title('Best HPI goat vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()

plt.plot(range(0,len(nwinlisthumanbagh)),nwinlisthumanbagh, label='Comp Bagh win rate')
plt.plot(range(0,len(pwinlisthumanbagh)),pwinlisthumanbagh, label='Comp Bagh lose rate')
plt.plot(range(0,len(drawlisthumanbagh)),drawlisthumanbagh, label='Comp Bagh draw rate')
plt.legend(loc='upper left')
plt.title('Best HPI tiger vs base(0) & best five (1-5) self-play agents')
plt.ylabel('Probability')
plt.xlabel('1st accepted followed by best 5 accepted agent')
plt.show()


In [ ]:
#Play a game against the agent
game=Game()
baghnet=PolicyValueNet(os.path.join(FILTERPATH, 'acceptedbagh_0.h5'))
goatnet=PolicyValueNet(os.path.join(HUMANPATH, 'best_goat.h5'))
goat=MCTSPlayer(goatnet.policy_value_fn, cpuct=5, n_playout=20)
bagh=MCTSPlayer(baghnet.policy_value_fn, cpuct=5, n_playout=20)
# player=HumanPlayer()

#change the players here to play as goat and tiger
data=game.start_play(goat,bagh)

# if you want to train further with human games
# data=[x for x in data]
# data=symmetry_board_moves(data)
# state_batch = [x[0] for x in data]
# mcts_probs_batch = [x[1] for x in data]
# winner_batch = [x[2] for x in data]
# pvnet.train(state_batch,mcts_probs_batch,winner_batch,5)
# pvnet.save_model("model.h5")